Dependencies



In [ ]:
import numpy as np
import pandas as pd
from itertools import permutations

# google maps api
import googlemaps 
gmaps = googlemaps.Client(key='/') 

# data
df = pd.read_csv('secretsanta.csv', index_col='name')

Functions to find the distance and optimal route

In [ ]:
# function to find the total distance from a list of names
def find_distance(names):
  max_distance = 0
  for i in range(len(names)-1):
    # distance between two postal codes
    distance = gmaps.distance_matrix(df.loc[names[i]]['code'],df.loc[names[i+1]]['code'])['rows'][0]['elements'][0]
    max_distance += distance['distance']['value']
  return max_distance

# function to find the optimal pick up route
def pick_up(perms):
  # checking each permutation for the number of gifts dropped off, and
  # recording the optimal permutation based on the max
  opt_stops = []
  remaining = []
  max_distance = 0
  for perm in perms:
    # stephanie is the first stop
    stops = ['Stephanie']
    # initializing remaining recipients
    perm_remaining = []
    for name in perm:
      perm_remaining.append(name)

    # iterating through each name in the permutation
    for i in range(len(perm)):
      # if gift giver is already in list from previous stop, drop off the gift
      for j in range(len(stops)):
        if df.loc[stops[j]]['recipient'] == perm[i]:
          perm_remaining.remove(perm[i])
      # add the name to the stops
      stops.append(perm[i])
    distance = find_distance(stops)
    (stops, distance) = drop_off(stops, distance, perm_remaining)
    # if the distance is minimized, this is the optimal pick up route
    if distance <= max_distance or max_distance == 0:
      print(stops, "  ", distance)
      max_distance = distance
      opt_stops = stops
      remaining = perm_remaining
  return (opt_stops, max_distance)

# function to find the optimal drop off route from the remaining names
def drop_off(stops, distance, remaining):
  # initialize the remaining names and their permutations
  remaining_perms = list(permutations(remaining))
  p = list(remaining_perms[0])
  p.insert(0, stops[len(stops)-1])
  p.append('Stephanie')
  max_remaining_distance = find_distance(p)
  opt_remaining_stops = p

  # find the optimal remaining drop off route
  for perm in remaining_perms:
    perm = list(perm)
    perm.insert(0, stops[len(stops)-1])
    perm.append('Stephanie')
    remaining_distance = find_distance(perm)
    if remaining_distance < max_remaining_distance:
      max_remaining_distance = remaining_distance
      opt_remaining_stops = perm

  # add the remaining stops and distance
  for i in range(1, len(opt_remaining_stops)):
    stops.append(opt_remaining_stops[i])

  distance += max_remaining_distance
  return (stops, distance)

Optimal route

In [ ]:
# finding the permutations of all names
names = []
for name in df.index:
  if name != 'Stephanie':
    names.append(name)
perms = list(permutations(names))

# finding the optimal route
(stops, distance) = pick_up(perms)

print("Route:")
for i in range(len(stops)):
  print(i+1, " ", stops[i])
print("Distance: ", distance/1000, " km")

['Stephanie', 'Florencia ', 'Neha', 'Megan', 'Alex', 'Lauryn', 'Janie', 'Megan', 'Alex', 'Florencia ', 'Stephanie']    58289
['Stephanie', 'Florencia ', 'Neha', 'Megan', 'Lauryn', 'Alex', 'Janie', 'Megan', 'Florencia ', 'Stephanie']    52816
['Stephanie', 'Florencia ', 'Neha', 'Megan', 'Lauryn', 'Janie', 'Alex', 'Megan', 'Florencia ', 'Stephanie']    52043
['Stephanie', 'Florencia ', 'Neha', 'Megan', 'Janie', 'Lauryn', 'Alex', 'Megan', 'Florencia ', 'Stephanie']    49706
['Stephanie', 'Florencia ', 'Neha', 'Lauryn', 'Alex', 'Megan', 'Janie', 'Megan', 'Florencia ', 'Stephanie']    49706
['Stephanie', 'Florencia ', 'Neha', 'Lauryn', 'Janie', 'Megan', 'Alex', 'Florencia ', 'Stephanie']    46133
['Stephanie', 'Florencia ', 'Neha', 'Janie', 'Megan', 'Lauryn', 'Alex', 'Florencia ', 'Stephanie']    45015
['Stephanie', 'Florencia ', 'Neha', 'Janie', 'Lauryn', 'Megan', 'Alex', 'Florencia ', 'Stephanie']    44211
['Stephanie', 'Neha', 'Megan', 'Lauryn', 'Alex', 'Janie', 'Florencia ', 'Florencia 